In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
tqdm.pandas()
from shapely.geometry import Point
from sklearn.neighbors import NearestNeighbors
import CityHub
import glob

/tmp/ipykernel_9054/184408677.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ModuleNotFoundError: No module named 'CityHub'

In [ ]:
def crossref(df, gdf):
    """
    Link rows of a dataframe to polygons in a geodataframe.
    Dataframe must have columns 'LONGITUDE' and 'LATITUDE'.
    """
    shapes = gdf.geometry
    centroids = [x.centroid for x in shapes]
    centroids = np.array([(x.x, x.y) for x in centroids])
    nn = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(centroids)
    _, indices = nn.kneighbors(df[['LONGITUDE', 'LATITUDE']])

    id_poly = []
    i = 0
    for _, row in tqdm(df.iterrows(), total=len(df)):
        point = Point(row['LONGITUDE'], row['LATITUDE'])
        new_id = None
        for idx in indices[i]:
            if shapes[idx].contains(point):
                new_id = idx
                break
        id_poly.append(new_id)
        i += 1

    df['id_poly'] = id_poly
    return df

## São Paulo

In [3]:
day_range = ["2019-12-01", "2020-02-01"]

In [4]:
hour_mapper = {
    "A NOITE" : "21:00:00",
    "A TARDE" : "15:00:00",
    "DE MADRUGADA" : "03:00:00",
    "PELA MANHÃ" : "09:00:00",
    "EM HORA INCERTA" : None,
}

In [5]:
def get_all_dates(day_range):
    dates = pd.date_range(day_range[0], day_range[1], freq="D")
    # repeat with hours: 3h, 9h, 15h, 18h
    new_dates = []
    for d in dates:
        new_dates.append(d + pd.Timedelta(hours=3))
        new_dates.append(d + pd.Timedelta(hours=9))
        new_dates.append(d + pd.Timedelta(hours=15))
        new_dates.append(d + pd.Timedelta(hours=21))
    return new_dates

def get_all_dates_2(day_range):
    dates = pd.date_range(day_range[0], day_range[1], freq="D")
    # repeat with hours: 3h, 9h, 15h, 18h
    new_dates = []
    for d in dates:
        new_dates.append(d + pd.Timedelta(hours=0))
        new_dates.append(d + pd.Timedelta(hours=12))
    return new_dates

In [6]:
def simplify_date(x):
    hour = x.hour
    if 6 <= hour < 12:
        new_hour = 9
    elif 12 <= hour < 18:
        new_hour = 15
    elif 18 <= hour < 24:
        new_hour = 21
    else:
        new_hour = 3
    return pd.to_datetime(f"{x.date()} {new_hour}:00:00")

def simplify_date_2(x):
    hour = x.hour
    if 6 <= hour < 18:
        new_hour = 12
    else:
        new_hour = 0
    return pd.to_datetime(f"{x.date()} {new_hour}:00:00")

In [7]:
poly_division = "SpCenterCensus2k"

### Waze

In [8]:
N = 10000000
df = pd.read_csv("data/time_series/waze-alerts.csv")
df = df[["geo", "ts", " type"]]
df = df.rename(columns = {" type" : "type"})

In [9]:
df["date"] = pd.to_datetime(df["ts"])

In [10]:
df = df[df.date >= day_range[0]]
df = df[df.date < day_range[1]]

In [11]:
df["LONGITUDE"] = df["geo"].apply(lambda x : x.split("(")[1].split(" ")[0])
df["LATITUDE"] = df["geo"].apply(lambda x : x.split(" ")[1].split(")")[0])
df.LONGITUDE = df.LONGITUDE.astype(float)
df.LATITUDE = df.LATITUDE.astype(float)
df["type"] = df["type"].apply(lambda x : "ROADCLOSED" if x == "ROAD_CLOSED" else x)
df = df[df.date >= day_range[0]]
df = df[df.date < day_range[1]]

In [13]:
gdf = gpd.read_file(f"data/shapefiles/{poly_division}.geojson")
df = crossref(df, gdf)
df = df.dropna(subset = ["id_poly"])

/home/giovani/anaconda3/envs/wavelet_code/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
100%|██████████| 1686772/1686772 [02:48<00:00, 9995.70it/s] 


In [14]:
df["updated_date"] = df["date"].apply(simplify_date)
df["id_poly"] = df["id_poly"].astype(int)
n_poly = len(gdf)
all_dates = get_all_dates(day_range)
n_days = len(all_dates)

for t in df.type.unique():
    df_ = df[df.type == t].copy()
    df_ = df_.groupby(["id_poly", "updated_date"]).size().reset_index()
    ts = np.zeros((n_poly, n_days))
    for i, d in enumerate(all_dates):
        filtered = df_[df_.updated_date == d]
        if len(filtered) == ts.shape[0]:
            ts[:, i] = filtered[0]
        else:
            ts[filtered.id_poly, i] = filtered[0]
    
    area = gdf.to_crs("EPSG:6933").area.values / 10**6
    ts = ts / area[:, None]
    np.save(f"data/time_series/{t}_{poly_division}_Period1.npy", ts)

In [15]:
df["updated_date"] = df["date"].apply(simplify_date_2)
df["id_poly"] = df["id_poly"].astype(int)
n_poly = len(gdf)
all_dates = get_all_dates_2(day_range)
n_days = len(all_dates)

for t in df.type.unique():
    df_ = df[df.type == t].copy()
    df_ = df_.groupby(["id_poly", "updated_date"]).size().reset_index()
    ts = np.zeros((n_poly, n_days))
    for i, d in enumerate(all_dates):
        filtered = df_[df_.updated_date == d]
        if len(filtered) == ts.shape[0]:
            ts[:, i] = filtered[0]
        else:
            ts[filtered.id_poly, i] = filtered[0]
    
    area = gdf.to_crs("EPSG:6933").area.values / 10**6
    ts = ts / area[:, None]
    np.save(f"data/time_series/{t}_{poly_division}_Period2.npy", ts)

### Crime theft and robbery

In [16]:
def load_theft():
    df = pd.read_csv("data/time_series/furto_celular_2018_2022.csv", sep = ";")
    df["hour"] = df.PERIDOOCORRENCIA.apply(lambda x : hour_mapper[x])
    df["date"] = pd.to_datetime(df.DATAOCORRENCIA, format='%d/%m/%Y', errors='coerce')
    df["date"] = df["date"] + pd.to_timedelta(df["hour"], errors='coerce')
    print(f"Fraction of nan dates: {df.date.isna().mean():.2f}")
    df = df.dropna(subset = ["date"])
    df = df[["date", "LATITUDE", "LONGITUDE"]]
    gdf = gpd.read_file(f"data/shapefiles/{poly_division}.geojson")
    df = crossref(df, gdf)
    df = df.dropna(subset = ["id_poly"])
    df["id_poly"] = df["id_poly"].astype(int)
    return df

In [17]:
gdf = gpd.read_file(f"data/shapefiles/{poly_division}.geojson")
df = load_theft()
df = df[df.date >= day_range[0]]
df = df[df.date < day_range[1]]
n_poly = len(gdf)
all_dates = get_all_dates(day_range)
n_days = len(all_dates)
df = df.groupby(["id_poly", "date"]).size().reset_index()

ts = np.zeros((n_poly, n_days))
for i, d in enumerate(all_dates):
    filtered = df[df.date == d]
    if len(filtered) == ts.shape[0]:
        ts[:, i] = filtered[0]
    else:
        ts[filtered.id_poly, i] = filtered[0]

area = gdf.to_crs("EPSG:6933").area.values / 10**6
ts = ts / area[:, None]
np.save(f"data/time_series/FurtoCelular_{poly_division}_Period1.npy", ts)

/tmp/ipykernel_707069/2194761068.py:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/time_series/furto_celular_2018_2022.csv", sep = ";")


Fraction of nan dates: 0.41


/home/giovani/anaconda3/envs/wavelet_code/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
100%|██████████| 239651/239651 [00:25<00:00, 9239.66it/s]


In [18]:
gdf = gpd.read_file(f"data/shapefiles/{poly_division}.geojson")
df = load_theft()
df = df[df.date >= day_range[0]]
df = df[df.date < day_range[1]]
df["date"] = df["date"].apply(simplify_date_2)

n_poly = len(gdf)
all_dates = get_all_dates_2(day_range)
n_days = len(all_dates)
df = df.groupby(["id_poly", "date"]).size().reset_index()

ts = np.zeros((n_poly, n_days))
for i, d in enumerate(all_dates):
    filtered = df[df.date == d]
    if len(filtered) == ts.shape[0]:
        ts[:, i] = filtered[0]
    else:
        ts[filtered.id_poly, i] = filtered[0]

area = gdf.to_crs("EPSG:6933").area.values / 10**6
ts = ts / area[:, None]
np.save(f"data/time_series/FurtoCelular_{poly_division}_Period2.npy", ts)

/tmp/ipykernel_707069/2194761068.py:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/time_series/furto_celular_2018_2022.csv", sep = ";")


Fraction of nan dates: 0.41


/home/giovani/anaconda3/envs/wavelet_code/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
100%|██████████| 239651/239651 [00:26<00:00, 9151.02it/s]


### Crime robbery

In [19]:
def load_robbery():
    df = pd.read_csv("data/time_series/roubo_celular_2018_2022.csv", sep = ";")
    df["hour"] = df.PERIDOOCORRENCIA.apply(lambda x : hour_mapper[x])
    df["date"] = pd.to_datetime(df.DATAOCORRENCIA, format='%d/%m/%Y', errors='coerce')
    df["date"] = df["date"] + pd.to_timedelta(df["hour"], errors='coerce')
    print(f"Fraction of nan dates: {df.date.isna().mean():.2f}")
    df = df.dropna(subset = ["date"])
    df = df[["date", "LATITUDE", "LONGITUDE"]]
    gdf = gpd.read_file(f"data/shapefiles/{poly_division}.geojson")
    df = crossref(df, gdf)
    df = df.dropna(subset = ["id_poly"])
    df["id_poly"] = df["id_poly"].astype(int)
    return df


In [20]:
gdf = gpd.read_file(f"data/shapefiles/{poly_division}.geojson")
df = load_robbery()
df = df[df.date >= day_range[0]]
df = df[df.date < day_range[1]]
n_poly = len(gdf)
n_poly = len(gdf)
all_dates = get_all_dates(day_range)
n_days = len(all_dates)
df = df.groupby(["id_poly", "date"]).size().reset_index()
ts = np.zeros((n_poly, n_days))
for i, d in enumerate(all_dates):
    filtered = df[df.date == d]
    if len(filtered) == ts.shape[0]:
        ts[:, i] = filtered[0]
    else:
        ts[filtered.id_poly, i] = filtered[0]

area = gdf.to_crs("EPSG:6933").area.values / 10**6
ts = ts / area[:, None]
np.save(f"data/time_series/RouboCelular_{poly_division}_Period1.npy", ts)

/tmp/ipykernel_707069/1937086392.py:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/time_series/roubo_celular_2018_2022.csv", sep = ";")


Fraction of nan dates: 0.01


/home/giovani/anaconda3/envs/wavelet_code/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
100%|██████████| 623682/623682 [01:04<00:00, 9618.97it/s]


In [21]:
gdf = gpd.read_file(f"data/shapefiles/{poly_division}.geojson")
df = load_robbery()
df = df[df.date >= day_range[0]]
df = df[df.date < day_range[1]]
df["date"] = df["date"].apply(simplify_date_2)
n_poly = len(gdf)
all_dates = get_all_dates_2(day_range)
n_days = len(all_dates)
df = df.groupby(["id_poly", "date"]).size().reset_index()
ts = np.zeros((n_poly, n_days))
for i, d in enumerate(all_dates):
    filtered = df[df.date == d]
    if len(filtered) == ts.shape[0]:
        ts[:, i] = filtered[0]
    else:
        ts[filtered.id_poly, i] = filtered[0]

area = gdf.to_crs("EPSG:6933").area.values / 10**6
ts = ts / area[:, None]
np.save(f"data/time_series/RouboCelular_{poly_division}_Period2.npy", ts)

/tmp/ipykernel_707069/1937086392.py:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/time_series/roubo_celular_2018_2022.csv", sep = ";")


Fraction of nan dates: 0.01


/home/giovani/anaconda3/envs/wavelet_code/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
100%|██████████| 623682/623682 [01:05<00:00, 9542.87it/s]


### Unify time-series

In [22]:
features_naming = {
    "JAM": "Jam",
    "ACCIDENT": "Accident",
    "ROADCLOSED": "Road Closed",
    "WEATHERHAZARD": "Weather Hazard",
    "FurtoCelular": "Phone Theft",
    "RouboCelular": "Phone Robbery"
}

In [23]:
ts = [np.load(f"data/time_series/{key}_{poly_division}_Period1.npy") for key in features_naming.keys()]
ts_sum = [np.sum(t) for t in ts]
ts = [t for i, t in enumerate(ts) if ts_sum[i] > 0]
signals = [s for i, s in enumerate(features_naming.keys()) if ts_sum[i] > 0]
date = get_all_dates(day_range)
df = []
for poly in range(ts[0].shape[0]):
    for t in range(ts[0].shape[1]):
        df.append({
            "date" : date[t],
            "id_poly" : poly,
        })
        for i, signal in enumerate(signals):
            df[-1][
                features_naming[signal]
            ] = ts[i][poly, t]

df = pd.DataFrame(df)
print(df.shape)
df.to_csv(f"data/polygon_data/{poly_division}_Period1.csv", index=False)

(504000, 8)


In [24]:
ts = [np.load(f"data/time_series/{key}_{poly_division}_Period2.npy") for key in features_naming.keys()]
ts_sum = [np.sum(t) for t in ts]
ts = [t for i, t in enumerate(ts) if ts_sum[i] > 0]
signals = [s for i, s in enumerate(features_naming.keys()) if ts_sum[i] > 0]
date = get_all_dates_2(day_range)
df = []
for poly in range(ts[0].shape[0]):
    for t in range(ts[0].shape[1]):
        df.append({
            "date" : date[t],
            "id_poly" : poly,
        })
        for i, signal in enumerate(signals):
            df[-1][
                features_naming[signal]
            ] = ts[i][poly, t]

df = pd.DataFrame(df)
print(df.shape)
df.to_csv(f"data/polygon_data/{poly_division}_Period2.csv", index=False)

(252000, 8)


## NY

In [3]:
month = "april"
df = pd.read_csv(f"data/time_series/2016_{month}_Yellow_Taxi_Trip_Data.csv")
df = df[df.trip_distance < 500]

In [4]:
df["LONGITUDE"] = df["pickup_longitude"]
df["LATITUDE"] = df["pickup_latitude"]
df["date"] = pd.to_datetime(df["tpep_pickup_datetime"])
df = df.drop(columns = [
    'VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
    'pickup_longitude', 'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag',
    'dropoff_longitude', 'dropoff_latitude', 'payment_type',  'extra', 
    'mta_tax', 'tolls_amount', 'improvement_surcharge', 'fare_amount', 'PULocationID', 'DOLocationID'], 
    axis = 1
)

/tmp/ipykernel_21394/2960374767.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["tpep_pickup_datetime"])


In [5]:
gdf = gpd.read_file("data/shapefiles/NYBlocks.geojson")
df = crossref(df, gdf)
df = df.dropna(subset = ["id_poly"])
df["id_poly"] = df["id_poly"].astype(int)

/home/giovani/anaconda3/envs/wavelet_code/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
100%|██████████| 11934328/11934328 [19:25<00:00, 10237.22it/s]


In [6]:
def get_all_dates(day_range):
    dates = pd.date_range(day_range[0], day_range[1], freq="D")
    # repeat with hours: 3h, 9h, 15h, 18h
    new_dates = []
    for d in dates:
        for h in range(2, 24, 4):
            new_dates.append(d + pd.Timedelta(hours=h))
        
    return new_dates

def simplify_date(x):
    hour = x.hour
    new_hour = hour // 4 * 4 + 2
    return pd.to_datetime(f"{x.date()} {new_hour}:00:00")

In [7]:
df["hour"] = df.date.dt.hour
df["hour"] = df.hour // 4 * 4 + 2
df["date"] = df["date"].dt.floor("d") + pd.to_timedelta(df["hour"], unit = "h")
df = df.groupby(["id_poly", "date"]
    ).agg(
        {"passenger_count" : ["mean", "sum"], "trip_distance" : "mean", "tip_amount" : "mean", "total_amount" : "mean", "date" : "count"}
    ).reset_index()
df.columns = ["id_poly", "date", "Passenger Count", "Total Passengers", "Trip Distance", "Tip Amount", "Total Amount", "N Pickups"]

In [8]:
day_range = ["2016-04-01", "2016-05-01"]

In [9]:
n_poly = len(gdf)
all_dates = get_all_dates(day_range)
n_days = len(all_dates)

columns = ["Passenger Count", "Total Passengers", "Trip Distance", "Tip Amount", "Total Amount", "N Pickups"]

ts = np.zeros((n_poly, n_days, 6))

for i, d in enumerate(all_dates):
    filtered = df[df.date == d]
    if len(filtered) == ts.shape[0]:
        ts[:, i, :] = filtered[columns]
    else:
    # fill missing values with 0
        ts[:, i] = 0
        ts[filtered.id_poly, i, :] = filtered[columns]

# change to a projection with meters as unit
area = gdf.to_crs("EPSG:6933").area.values / 10**6
ts = ts / area[:, None, None]


In [10]:
for i in range(len(columns)):
    np.save(f"data/time_series/{columns[i]}_NYBlocks_Period1.npy", ts[:, :, i])

### Unify time-series

In [11]:
poly_division = "NYBlocks"
time_interval = "Period1"
ts = [
    np.load(f"data/time_series/{signal}_{poly_division}_{time_interval}.npy") for signal in columns
]
# drop if all values are 0
ts_sum = [np.sum(t) for t in ts]
ts = [t for i, t in enumerate(ts) if ts_sum[i] > 0]
columns = [s for i, s in enumerate(columns) if ts_sum[i] > 0]
df = []
date = get_all_dates(day_range)
for poly in range(ts[0].shape[0]):
    for t in range(ts[0].shape[1]):
        df.append({
            "date" : date[t],
            "id_poly" : poly,
        })
        for i, signal in enumerate(columns):
            df[-1][signal] = ts[i][poly, t]

df = pd.DataFrame(df)
print(df.shape)
df.to_csv(f"data/polygon_data/{poly_division}_{time_interval}.csv", index=False)

(217248, 8)


## BLA Cities

In [60]:
gdf = gpd.read_file("data/shapefiles/BLACities.geojson")
name = gdf.name.tolist()
id_poly = gdf.id_poly.tolist()
df = pd.read_csv("data/time_series/cities_time_series.csv")
df = df.drop(columns = ['nm_municip', 'x_start', 'y_start', 'x_width', 'y_width'])

In [61]:
father_classes_hierarchy = {
    "floresta": [
        "formacao_florestal",
        "formacao_savanica",
        "mangue",
        "restinga_arborizada",
    ],
    "formacao_natural_nao_florestal": [
        "campo_alagado_e_area_pantanosa",
        "formacao_campestre",
        "apicum",
        "afloramento_rochoso",
        "outras_formacoes_nao_florestais",
    ],
    "agropecuaria": [
        "pastagem",
        "soja",
        "cana",
        "arroz",
        "outras_lavouras_temporarias",
        "cafe",
        "citrus",
        "outras_lavouras_perenes",
        "floresta_plantada",
        "moisaco_de_agricultura_e_pastagem",
    ],
    "area_nao_vegetada": [
        "praia_e_duna",
        "infraestrutura_urbana",
        "mineracao",
        "outras_areas_nao_vegetadas",
    ],
    "corpo_dagua": ["rio_lago_e_oceano", "aquicultura"],
    "nao_observado": [
        "nao_observado",  # "nao_classificado"
    ],
}


In [62]:
for father, children in father_classes_hierarchy.items():
    df[father] = df[children].sum(axis = 1)

In [63]:
selected_columns = ["formacao_florestal", "formacao_savanica", "pastagem", "soja", "cana", "mineracao", "infraestrutura_urbana", "moisaco_de_agricultura_e_pastagem", "mangue"]
features_naming = {
    "formacao_florestal" : "Forest Formation",
    "formacao_savanica" : "Cerrado",
    "pastagem" : "Pasture",
    "soja" : "Soy",
    "cana" : "Sugar Cane",
    "mineracao" : "Mining",
    "infraestrutura_urbana" : "Urban Infrastructure",
    "moisaco_de_agricultura_e_pastagem" : "Mosaic Agriculture",
    "mangue" : "Mangrove"
}

In [64]:
df = df[["year", "cd_geocmu"] + selected_columns]

In [65]:
def calculate_yearly_diff(df):
    df_copy = df.sort_values("year")
    for col in selected_columns:
        df_copy[col] = df[col].diff()
    df_copy = df_copy.dropna()
    return df_copy
    
df_diff = df.groupby("cd_geocmu").apply(calculate_yearly_diff, include_groups = False).reset_index()
df_diff = df_diff.drop(columns = ["level_1"])
df_diff["year"] = df_diff.year.apply(lambda x : int(x - 1))

In [66]:
df_result = []
for year in df.year.unique():
    df_ = df[df.year == year]
    new_df = {
        "id_poly" : id_poly,
    }
    for column in selected_columns:
        new_df[features_naming[column]] = df_[column]
    new_df = pd.DataFrame(new_df)
    new_df["date"] = pd.to_datetime(f"{year}-01-01")
    df_result.append(new_df)
df_result = pd.concat(df_result)

In [67]:
#place date and id_poly as first columns
cols = df_result.columns.tolist()
cols.remove("date")
cols.remove("id_poly")
cols = ["date", "id_poly"] + cols
df_result = df_result[cols]
df_result = df_result.sort_values(["id_poly", "date"])
df_result.to_csv("data/polygon_data/BLACities_Year.csv", index=False)

In [68]:
df_result = []
for year in df_diff.year.unique():
    df_ = df_diff[df_diff.year == year]
    new_df = {
        "id_poly" : id_poly,
    }
    for column in selected_columns:
        new_df[features_naming[column]] = df_[column]
    new_df = pd.DataFrame(new_df)
    new_df["date"] = pd.to_datetime(f"{year}-01-01")
    df_result.append(new_df)
df_result = pd.concat(df_result)

In [70]:
#place date and id_poly as first columns
cols = df_result.columns.tolist()
cols.remove("date")
cols.remove("id_poly")
cols = ["date", "id_poly"] + cols
df_result = df_result[cols]
df_result = df_result.sort_values(["id_poly", "date"])
df_result.to_csv("data/polygon_data/BLACities_Year2.csv", index=False)